In [36]:
import pandas as pd
import yfinance as yf

In [37]:
# Sample Inputs
total_investment = 1000  # Total investment amount
start_date = "2024-05-01"  # Investment period start date
end_date = "2024-05-05"  # Investment period end date


In [38]:
stocks = [
	{"Ticker": "ADANIPORTS.NS", "Weightage": 35.0000},
	{"Ticker": "APOLLOHOSP.NS", "Weightage": 30.0000},
	{"Ticker": "ASIANPAINT.NS", "Weightage": 35.0000},]


In [39]:
def calculate_rsi(data, periods=14):
	"""
	Calculates the Relative Strength Index (RSI) for a given DataFrame.

	Args:
		data: pandas DataFrame containing the 'Close' price data.
		periods: Number of periods for RSI calculation (default: 14).

	Returns:
		pandas Series containing the RSI values.
	"""

	delta = data["Close"].diff()
	gain = (delta.where(delta > 0, 0)).fillna(0)
	loss = (delta.where(delta < 0, 0)).fillna(0)

	avg_gain = gain.rolling(window=periods).mean()
	avg_loss = abs(loss.rolling(window=periods).mean())

	rs = avg_gain / avg_loss
	rsi = 100 - (100 / (1 + rs))

	print(rsi)

	return rsi

In [40]:
# Adjust weightage based on RSI
def adjust_weightage_by_rsi(weightage, rsi):
    if rsi > 90:
        return weightage - 0.003
    elif rsi > 80:
        return weightage - 0.002
    elif rsi > 70:
        return weightage - 0.001
    elif rsi < 20:
        return weightage + 0.003
    elif rsi < 30:
        return weightage + 0.002
    elif rsi <= 70:
        return weightage + 0.001
    return weightage


# Adjusted start date for RSI calculation
adjusted_start_date = pd.to_datetime(start_date) - pd.Timedelta(days=24)
end_date = pd.to_datetime(end_date)

# Daily investment (total investment divided by the number of days)
total_days = (end_date - pd.to_datetime(start_date)).days + 1
daily_investment = total_investment / total_days

# Results list for investment details
results = []

all_rsi = {}
for stock in stocks:
    ticker = stock["Ticker"]
    # Download stock data including extended range for RSI calculation
    data = yf.download(ticker, start=adjusted_start_date, end=end_date)
    if data.empty:
        continue
    # Calculate RSI
    data["RSI"] = calculate_rsi(data, periods=14)
    # Filter data to only include rows starting from the actual start date
    all_rsi[ticker] = data[data.index >= pd.to_datetime(start_date)]

# Loop through each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    if ticker not in all_rsi or all_rsi[ticker].empty:
        print(f"Data for {ticker} is not available. Skipping.")
        continue

    data = all_rsi[ticker]

    # Adjust weightage dynamically
    data["Adjusted Weightage"] = data["RSI"].apply(
        lambda x: adjust_weightage_by_rsi(weightage, x)
    )



    # Daily allocated investment based on normalized adjusted weightage
    data["Allocated Investment"] = (
        (data["Adjusted Weightage"] / 100) * total_investment
    )

    # Safely calculate shares (consider handling zero close prices and insufficient funds)
    data["Shares"] = data.apply(
        lambda row: (
            "Not enough money"
            if row["Close"].iloc[0] == 0
            or row["Allocated Investment"].iloc[0] < row["Close"].iloc[0]
            else row["Allocated Investment"].iloc[0] // row["Close"].iloc[0]
        ),
        axis=1,
    )

    # Append results with additional investment details
    for index, row in data.iterrows():
        results.append(
            {
                "Ticker": ticker,
                "Date": index.date(),
                "Weightage": weightage,
                "Adjusted Weightage": row["Adjusted Weightage"].iloc[0],
                "Allocated Investment": row["Allocated Investment"].iloc[0],
                "Open Price": row["Open"].iloc[0],
                "Close Price": row["Close"].iloc[0],
                "RSI": row["RSI"].iloc[0],
                "Shares": row["Shares"].iloc[0],
            }
        )

# Convert results to DataFrame
summary = pd.DataFrame(results)

# Reorder and display columns dynamically
columns = [
    "Ticker",
    "Date",
    "Weightage",
    "Adjusted Weightage",
    "Allocated Investment",
    "Open Price",
    "Close Price",
    "RSI",
    "Shares",
]
summary = summary[columns]

# Save the summary to CSV
summary.to_csv("investment_summary000.csv", index=False)

print("\nSummary saved to 'investment_summary.csv'")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Ticker      ADANIPORTS.NS
Date                     
2024-04-08            NaN
2024-04-09            NaN
2024-04-10            NaN
2024-04-12            NaN
2024-04-15            NaN
2024-04-16            NaN
2024-04-18            NaN
2024-04-19            NaN
2024-04-22            NaN
2024-04-23            NaN
2024-04-24            NaN
2024-04-25            NaN
2024-04-26            NaN
2024-04-29      36.949275
2024-04-30      40.719027
2024-05-02      43.395541
2024-05-03      39.211062



[*********************100%***********************]  1 of 1 completed


Ticker      APOLLOHOSP.NS
Date                     
2024-04-08            NaN
2024-04-09            NaN
2024-04-10            NaN
2024-04-12            NaN
2024-04-15            NaN
2024-04-16            NaN
2024-04-18            NaN
2024-04-19            NaN
2024-04-22            NaN
2024-04-23            NaN
2024-04-24            NaN
2024-04-25            NaN
2024-04-26            NaN
2024-04-29      38.719758
2024-04-30      38.180450
2024-05-02      29.605162
2024-05-03      32.544620
Ticker      ASIANPAINT.NS
Date                     
2024-04-08            NaN
2024-04-09            NaN
2024-04-10            NaN
2024-04-12            NaN
2024-04-15            NaN
2024-04-16            NaN
2024-04-18            NaN
2024-04-19            NaN
2024-04-22            NaN
2024-04-23            NaN
2024-04-24            NaN
2024-04-25            NaN
2024-04-26            NaN
2024-04-29      45.831869
2024-04-30      47.254419
2024-05-02      66.522281
2024-05-03      54.534214

Summary sav